In [24]:
#気象庁
#過去7日間の最高気温のデータを取得するコード
#https://www.data.jma.go.jp/stats/data/mdrr/docs/csv_dl_readme.html
#ウェブ上のデータは毎日5:00ごろ更新＋修正は13:00、19:00、翌日1:00ごろの更新で反映
#定期実行

import pandas as pd
import re

#データを取得してファイル名に日付を入れて保存する
def get_previous_data(d):
    url = f'https://www.data.jma.go.jp/obd/stats/data/mdrr/tem_rct/alltable/mxtemsadext0{d}.csv'
    data = pd.read_csv(url, encoding='Shift-JIS')

    #日付
    yyyymmdd = f"{data['観測時刻(年)'][0]}{data.loc[0,['観測時刻(月)','観測時刻(日)']].apply(lambda x: str(x).zfill(2)).sum()}"
    #ファイル名
    
    filename = f"./data-maxtemp/daily-data/jma-maxtemp-{yyyymmdd}.csv"
    #dataにも記載
    data['date'] = pd.to_datetime(yyyymmdd, format='%Y%m%d')
    #csvで出力
    data.to_csv(filename, index=False)
    return data

#実行
#集計（真夏・猛暑日を数える）
data_agg = pd.DataFrame()

###データ取得
for d in range(1, 8): 
    data = get_previous_data(d)
    ##ととのえる
    ##最高気温のデータのカラムを抽出
    ##（1日前＝昨日の~、2日前=一昨日の~、3日以上前=xx日の最高気温(℃)
    rename_dic = {col:'maxtemp' for col in data.columns if '日の最高気温(℃)' in col}
    data = data.rename(columns=rename_dic)
    ##統合
    data_agg = pd.concat([data_agg, data])

###計算
data_agg['over30'] = data_agg.maxtemp >= 30
data_agg['over35'] = data_agg.maxtemp >= 35
data_agg['over40'] = data_agg.maxtemp >= 40
data_agg['total'] = 1
data_agg['null_values'] = data_agg.maxtemp.isna()

cols = ['over30','over35','over40','total','null_values']
#集計df
heat_points_7days = data_agg.groupby(['date'])[cols].sum().reset_index()

#過去分と統合
filename = "./data-maxtemp/timeseries-data/jma-maxtemp-heatpoints-ts.csv"
##過去分を取得
heat_points = pd.read_csv(filename)
heat_points = pd.concat([heat_points,heat_points_7days])
heat_points = heat_points.reindex(['date']+cols, axis=1)
##重複削除（新しく入ったほうを残す）
heat_points.date = pd.to_datetime(heat_points.date)
heat_points = heat_points.loc[~heat_points.duplicated(subset=['date'], keep='last')].reset_index(drop=True)
heat_points = heat_points.dropna(subset='date').sort_values(by='date')

##出力
heat_points.to_csv(filename, index=False)

URLError: <urlopen error [Errno 60] Operation timed out>

修正で追加するコード

In [22]:
#観測地点の一覧
url = 'https://raw.githubusercontent.com/Nikkei-Visual-Data-Journalism/Heatwave/main/data-maxtemp/meta/points_list.csv'
points = pd.read_csv(url)
#dic{観測所番号:県名}
pref_dic = points.set_index('観測所番号').pref.to_dict()
#県庁所在地の観測所番号リスト
capitol = points[points.capitol==1]['観測所番号'].to_list()
#data_aggに県名、県庁所在地フラグを反映
data_agg['pref'] = data_agg['観測所番号'].map(pref_dic)
data_agg['capitol'] = None
data_agg.loc[data_agg['観測所番号'].isin(capitol),'capitol'] = 1
#日付、年
data_agg.date = pd.to_datetime(data_agg.date)
data_agg['year'] = data_agg.date.dt.year

In [25]:
data_agg

""


In [ ]:
#県庁所在地の最高温度をつけたす
#capitol = data_agg[data_agg.capitol==1].set_index(['date','year','pref']).maxtemp.rename('maxtemp_capitol')
#df_count = pd.concat([df_count, capitol],axis=1)